# Experimental Evaluation: Goal Recognition in FOND with Temporally Extended Goals (LTLf and PLTL)

In [198]:
BASE_DIR = '../fond-recognition-benchmarks'

In [199]:
import os
import sys
import pandas as pd
import glob
import json
import numpy as np
import itertools

In [200]:
def compute_posterior(index, prob, priors):
    num = prob[index] * priors[index]
    denom = np.sum(prob*priors)
    return num / denom

In [201]:
RESULTS_DIR = os.path.join(BASE_DIR, 'results/')

In [202]:
domains_evaluation = ['blocksworld', 'logistics', 'tidyup', 'tireworld', 'triangle-tireworld', 'zenotravel']

json_files = []
for domain in domains_evaluation:
    files = glob.glob(os.path.join(RESULTS_DIR + '/' + domain, '*.json'))
    for f in files:
        json_files.append(f)

In [203]:
print("Results found:", len(json_files))

Results found: 780


# Results Collection

In [204]:
print(json_files[0])
print(os.path.basename(json_files[0]))
filename, ext = os.path.basename(json_files[0]).split('.')

../fond-recognition-benchmarks/results//blocksworld/blocks-world_p01_hyp-3_full.json
blocks-world_p01_hyp-3_full.json


In [205]:
tokens = filename.split('_')

In [206]:
dataset = {
    'domain' : [],\
    'problem' : [],\
    'observability' : [],\
    'goals' : [],\
    'judge_point' : [],\
    'obs_len' : [],\
    'true_goal': [],\
    'time': [],\
    'posterior': []}

sum_obs_dict = dict()
problems_obs_dict = dict()

for pathname in json_files:
    filename, ext = os.path.basename(pathname).split('.')
    tokens = filename.split('_')
    approach = tokens[0]
    model_type = tokens[-1]
    data = {}
    with open(pathname) as instream:
        buffer = instream.read()
        data = json.loads(buffer)
    #print(approach, model_type, data['domain'], data['problem'], data['observability'])
    num_goals = len(data["G"])
    true_goal = data["G"].index(data['G*'])
    likelihoods = data['P(Obs | G)']
    
    if data['observability'] in sum_obs_dict:
        problems_obs_dict[data['observability']] += 1
        sum_obs_dict[data['observability']] += len(data['Obs'])
    else:
        problems_obs_dict[data['observability']] = 1
        sum_obs_dict[data['observability']] = len(data['Obs'])
    
    for k, prob_O_G in enumerate(likelihoods):
        post_probs = [compute_posterior(j, prob_O_G, np.ones(num_goals)/num_goals) \
                     for j in range(len(data['G']))]
        dataset['domain'] += [data['domain']]
        dataset['problem'] += [data['problem']]
        dataset['goals'] += [num_goals]
        dataset['judge_point'] += [k]
        dataset['obs_len'] += [len(data['Obs'])]
        dataset['true_goal'] += [true_goal]
        dataset['observability'] += [data['observability']]
        dataset['time'] += [data['time']]
        dataset['posterior'] += [np.array(post_probs)]

avg_obs = 0.0
sum_obs = 0.0
avg_obs_observability = dict()
for k in sum_obs_dict.keys():
    avg = sum_obs_dict[k] / problems_obs_dict[k]
    avg_obs_observability[k] = avg
    sum_obs += avg
    
avg_obs = sum_obs / len(sum_obs_dict)

In [207]:
dataset = pd.DataFrame(dataset)

In [208]:
def compute_accuracy(dataset, top_k):
    tp_count = 0
    for index, row in dataset.iterrows():
        top_k_goals = np.argpartition(row['posterior'], -top_k)[-top_k:]
        if row['true_goal'] in top_k_goals:
            tp_count += 1
    return tp_count / len(dataset)

In [209]:
def compute_recognition_time(dataset):
    sum_time = 0
    for index, row in dataset.iterrows():
        sum_time += row['time']

    return (sum_time / len(dataset))

In [210]:
def compute_error_rates(dataset):
    tpr = 0.0
    fpr = 0.0
    fnr = 0.0

    for index, row in dataset.iterrows():
        tp_count = 0
        tn_count = 0
        fp_count = 0
        fn_count = 0

        top_goal = np.argmax(row['posterior'])
        max_goals = [ k for k, p in enumerate(row['posterior']) if p == row['posterior'][top_goal]]
        
        if row['true_goal'] in max_goals:
            tp_count = 1
        fn_count = 1 - tp_count
        fp_count = len(max_goals) - tp_count
        tn_count = row['goals'] - fp_count
    
        tpr += tp_count/(tp_count + fp_count)
        fpr += fp_count/(fp_count + tn_count)
        fnr += fn_count/(fn_count + tp_count)
    return tpr/len(dataset), fpr/len(dataset), fnr/len(dataset)

## Online Goal Recognition

In [211]:
approaches = ['approach']
observabilities = ['10', '25', '30', '50', '70', '75', '100']

summary = {'approach': [], 'observability': [], 'top1': [], 'top2': [], 'top3': [], 'tpr' : [], 'fpr':[], 'fnr': []}

for approach, observability in itertools.product(approaches, observabilities):
    df = dataset.loc[(dataset['observability'] == observability)]
    if len(df) == 0: 
        continue
    top_1_acc = compute_accuracy(df, 1)
    top_2_acc = compute_accuracy(df, 2)
    top_3_acc = compute_accuracy(df, 3)
    tpr, fpr, fnr = compute_error_rates(df)
    summary['approach'] += [approach]
    summary['observability'] += [observability]
    summary['top1'] += [top_1_acc]
    summary['top2'] += [top_2_acc]
    summary['top3'] += [top_3_acc]
    summary['tpr'] += [tpr]
    summary['fpr'] += [fpr]
    summary['fnr'] += [fnr]

In [212]:
summary = pd.DataFrame(summary)

In [213]:
summary[['observability', 'top1', 'top2', 'top3', 'tpr', 'fpr', 'fnr']]

,observability,top1,top2,top3,tpr,fpr,fnr
0,10,0.514894,0.730496,0.843972,0.512543,0.135177,0.432624
1,30,0.505641,0.733333,0.860513,0.504481,0.136568,0.435385
2,50,0.503311,0.740460,0.853359,0.499616,0.137993,0.439294
3,70,0.499324,0.725000,0.850450,0.496953,0.139687,0.438063
4,100,0.498077,0.727404,0.848558,0.496430,0.139251,0.438942


## Offline Goal Recognition

In [214]:
offline = {'observability': [], 'avg_obs': [], 'time': [], 'top1': [], 'top2': [], 'top3': [], 'tpr' : [], 'fpr':[], 'fnr': []}

for approach, observability in itertools.product(approaches, observabilities):
    df = dataset.loc[(dataset['observability'] == observability)\
                    & (dataset['judge_point'] == dataset['obs_len'] - 1)]
    if len(df) == 0: continue
    top_1_acc = compute_accuracy(df, 1)
    top_2_acc = compute_accuracy(df, 2)
    top_3_acc = compute_accuracy(df, 3)
    tpr, fpr, fnr = compute_error_rates(df)
    avg_time = compute_recognition_time(df)
    offline['observability'] += [observability]
    offline['avg_obs'] += [avg_obs_observability[observability]]
    offline['time'] += [avg_time]
    offline['top1'] += [top_1_acc]
    offline['top2'] += [top_2_acc]
    offline['top3'] += [top_3_acc]
    offline['tpr'] += [tpr]
    offline['fpr'] += [fpr]
    offline['fnr'] += [fnr]

In [215]:
offline = pd.DataFrame(offline)
offline[['observability', 'avg_obs', 'time', 'top1', 'top2', 'top3', 'tpr', 'fpr', 'fnr']]

,observability,avg_obs,time,top1,top2,top3,tpr,fpr,fnr
0,10,3.916667,243.905876,0.655556,0.844444,0.927778,0.657593,0.112778,0.255556
1,30,10.833333,235.070979,0.744444,0.877778,0.927778,0.750000,0.101343,0.138889
2,50,17.616667,242.068927,0.788889,0.944444,0.972222,0.797222,0.077593,0.105556
3,70,24.666667,232.130767,0.788889,0.905556,0.933333,0.818981,0.083796,0.072222
4,100,34.666667,272.847798,0.800000,0.933333,0.966667,0.829167,0.086111,0.050000


In [216]:
print('Average observations: %s' % avg_obs)

Average observations: 18.34


### First Observation

In [217]:
first_obs = {'observability': [], 'top1': [], 'top2': [], 'top3': [], 'tpr' : [], 'fpr':[], 'fnr': []}

for approach, observability in itertools.product(approaches, observabilities):
    df = dataset.loc[(dataset['observability'] == observability)\
                    & (dataset['judge_point'] == 0)]
    if len(df) == 0: continue
    #print(len(df))
    top_1_acc = compute_accuracy(df, 1)
    top_2_acc = compute_accuracy(df, 2)
    top_3_acc = compute_accuracy(df, 3)
    tpr, fpr, fnr = compute_error_rates(df)
    first_obs['observability'] += [observability]
    first_obs['top1'] += [top_1_acc]
    first_obs['top2'] += [top_2_acc]
    first_obs['top3'] += [top_3_acc]
    first_obs['tpr'] += [tpr]
    first_obs['fpr'] += [fpr]
    first_obs['fnr'] += [fnr]

In [218]:
first_obs = pd.DataFrame(first_obs)
first_obs[['observability', 'top1', 'top2', 'top3', 'tpr', 'fpr', 'fnr']]

,observability,top1,top2,top3,tpr,fpr,fnr
0,10,0.322222,0.550000,0.750000,0.329683,0.192593,0.583333
1,30,0.211111,0.450000,0.672222,0.218717,0.223981,0.677778
2,50,0.244444,0.411111,0.616667,0.235198,0.225741,0.666667
3,70,0.205556,0.377778,0.577778,0.210000,0.222963,0.711111
4,100,0.200000,0.383333,0.550000,0.201667,0.221111,0.716667
